In [1]:
import os, openai
import dotenv
from langchain.tools import Tool
from dotenv import load_dotenv

from langchain.chains import SequentialChain, LLMChain
from langchain.agents import initialize_agent, Tool
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from googleapiclient.discovery import build
from langchain.chains.base import Chain

dotenv.load_dotenv('../.env')
GOOGLE_API_KEY = os.getenv("GOOGLE-SEARCH-API-KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH-ENGINE-ID")

openai.api_key = os.getenv("CHATGPT-RECEIPT")

In [5]:
def search_with_google_api(query, num_results=5):
    api_key = GOOGLE_API_KEY  # Google API 키
    cse_id = SEARCH_ENGINE_ID   # Custom Search Engine ID
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        results = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
        items = results.get("items", [])
        return [
            {
                "title": item["title"], "snippet": item.get("snippet")  # 본문 내용 추가
            } 
            for item in items
        ]
    except Exception as e:
        return f'Google Search API Error: {e}'


class SearchChain(Chain):
    def __init__(self, tool):
        super().__init__()
        self._tool = tool

    @property
    def input_keys(self):
        return ["business_name"]

    @property
    def output_keys(self):
        return ["search_results"]

    def _call(self, inputs, **kwargs):
        business_name = inputs["business_name"]
        search_results = self._tool.func(business_name)
        return {"search_results": search_results}


# Tool로 검색 기능 정의
search_tool = Tool(
    name = "SearchBusinessCategory",
    func = lambda business_name: "\n".join(
        list([x['snippet'] for x in search_with_google_api(business_name)])
    ),
    description="상호명을 검색하여 관련 업종 정보를 제공합니다."
)

# Prompt 정의
analyze_prompt = PromptTemplate(
    input_variables=["business_name", "search_results"],
    template="""
    다음은 상호명 '{business_name}'에 대한 검색 결과입니다:
    {search_results}
    검색 결과를 기반으로 해당 상호의 업종을 추론하세요. 가능한 경우 단답형으로 업종만 답변하세요.
    예: "식당", "카페", "의류", "교통" 등. 
    """
)

# Search chain
search_chain = SearchChain(search_tool)


# 검색 결과를 분석하는 LLMChain
analyze_chain = LLMChain(
    llm = ChatOpenAI(
        model="gpt-4", 
        temperature=0,
        openai_api_key = openai.api_key
    ),
    prompt=analyze_prompt,
    output_key="business_category",  # 출력 키를 명시적으로 설정
)

In [13]:
sequential_chain = SequentialChain(
    chains=[search_chain, analyze_chain],
    input_variables=["business_name"],
    output_variables=["business_category"],
    verbose=True
)

business_name = "waveon 기장"
response = sequential_chain.invoke({"business_name": business_name})
print(response)




> Entering new SequentialChain chain...

> Finished chain.
{'business_name': 'waveon 기장', 'business_category': '카페'}
